In [3]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [7]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_97ff5280ec39471bbfab852056f760de = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-mUqK-Yod9nR5R05qd2iyDwvdViEH8yUDUr4Rs8ijO8Q',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_97ff5280ec39471bbfab852056f760de.get_object(Bucket='decisiontreedrugrecommendation-donotdelete-pr-3ghbh8pzu5cuju',Key='drug200.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

my_data = pd.read_csv(body)
my_data.head()


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


In [8]:
my_data.columns

Index(['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K', 'Drug'], dtype='object')

In [9]:
X = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values

#Convert Categorical Value to Numerical Value

In [10]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
X[:,1] = le_sex.transform(X[:,1]) 

In [11]:
le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X[:,2] = le_BP.transform(X[:,2])

In [12]:
le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
X[:,3] = le_Chol.transform(X[:,3]) 

In [13]:
y=my_data["Drug"]

#Split Data-Train-Test

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=.3)

#Modeling

In [15]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)

In [16]:
drugTree.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

#Prediction

In [17]:
Y_hat=drugTree.predict(X_test)

In [18]:
Y_hat

array(['drugX', 'drugY', 'drugX', 'drugY', 'drugC', 'drugY', 'drugY',
       'drugY', 'drugA', 'drugY', 'drugY', 'drugX', 'drugY', 'drugY',
       'drugY', 'drugX', 'drugY', 'drugX', 'drugB', 'drugY', 'drugC',
       'drugY', 'drugA', 'drugA', 'drugA', 'drugA', 'drugY', 'drugY',
       'drugC', 'drugY', 'drugC', 'drugY', 'drugY', 'drugB', 'drugX',
       'drugX', 'drugX', 'drugY', 'drugY', 'drugC', 'drugY', 'drugY',
       'drugX', 'drugY', 'drugY', 'drugX', 'drugY', 'drugB', 'drugY',
       'drugB', 'drugX', 'drugA', 'drugX', 'drugY', 'drugY', 'drugY',
       'drugY', 'drugX', 'drugX', 'drugX'], dtype=object)

Evaluation

In [19]:
from sklearn import metrics
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(Y_test, Y_hat))

DecisionTrees's Accuracy:  0.9833333333333333


In [ ]:
from sklearn.externals.six import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

In [ ]:
dot_data = StringIO()
filename = "drugtree.png"
featureNames = my_data.columns[0:5]
targetNames = my_data["Drug"].unique().tolist()
out=tree.export_graphviz(drugTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_trainset), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')